In [1]:
%cd ../

/Users/eloidieme/dev/python-projects/convnet


In [2]:
import numpy as np
from cnnClassifier.data.make_dataset import DataLoader

np.random.seed(400)

### Load and split the data

In [3]:
load = DataLoader()
data = load.make_data()

In [4]:
data_file = np.load('data/train_val_data.npz')

In [5]:
X_train = data_file['X_train']
Y_train = data_file['Y_train']
y_train = data_file['y_train']
X_val = data_file['X_val']
Y_val = data_file['Y_val']
y_val = data_file['y_val']

### Network hyper-parameters

In [6]:
n1 = 5  # number of filters at layer 1
k1 = 5  # width of the filters at layer 1 (final shape: d x k1)
n2 = 5  # number of filters at layer 2
k2 = 5  # width of the filters at layer 1 (final shape: d x k2)
eta = 0.001  # learning rate
rho = 0.9  # momentum term

d = load.meta['dimensionality']  # dimensionality
K = load.meta['n_classes']  # number of classes
n_len = load.meta['n_len']  # max len of a name

n_len1 = n_len - k1 + 1
n_len2 = n_len1 - k2 + 1

f_size = n2 * n_len2

### Parameters initialization

In [7]:
# USE HE INIT FOR SIGMAS
sigma1 = 0.3
sigma2 = 0.3
sigma3 = 0.3

F = []
F.append(np.random.normal(0.0, sigma1, (d, k1, n1)))
F.append(np.random.normal(0.0, sigma2, (n1, k2, n2)))
W = np.random.normal(0.0, sigma3, (K, f_size))

In [8]:
X_train.shape

(532, 19834)

In [9]:
Y_train.shape

(18, 19834)

In [20]:
x_input = X_train[:,0]
x_input

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [21]:
X_input = np.reshape(x_input, (d, -1))
X_input

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0,

### MF matrix implementation

In [14]:
def make_mf_matrix(F, n_len):
    dd, k, nf = F.shape
    V_F = []
    for i in range(nf):
        V_F.append(F[:,:,i].ravel(order='F').T)
    V_F = np.array(V_F)
    zero_nlen = np.zeros((dd, nf))
    kk = n_len - k
    Mf = []
    for i in range(kk+1):
        tup = [zero_nlen.T for _ in range(kk + 1)]
        tup[i] = V_F
        Mf.append(np.concatenate(tup, axis=1))
    Mf = np.concatenate(Mf, axis=0)
    return Mf

In [18]:
for i in range(2):
    dd, k, nf = F[i].shape
    assert ((n_len-k+1)*nf, n_len*dd) == make_mf_matrix(F[i], n_len).shape

### MX matrix implementation

In [51]:
def make_mx_matrix(X_input, d, k, nf, n_len):
    I_nf = np.eye(nf)
    Mx = []
    for i in range(n_len - k + 1):
        Mx.append(np.kron(I_nf, X_input[:d,i:i+k].ravel(order='F').T))
    Mx = np.concatenate(Mx, axis=0)
    return Mx

In [25]:
for i in range(2):
    d, k, nf = F[i].shape
    assert ((n_len - k + 1)*nf, k*nf*d) == make_mx_matrix(X_input, d, k, nf, n_len).shape
make_mx_matrix(x_input, d, k, nf, n_len).shape

(75, 125)

### Convolution matrices test

In [54]:
d, k, nf = F.shape
MX = make_mx_matrix(X_input, d, k, nf, n_len)
MF = make_mf_matrix(F, n_len)
s1 = MX @ np.ravel(F, order='F')
s2 = MF @ x_input
np.all(s1 == s2)

True

In [59]:
F[1].shape

(5, 5, 5)

In [53]:
X = X_input
x = np.ravel(X, order='F')
F = np.random.normal(0, 0.2, (28, 2, 4))
MF = make_mf_matrix(F, n_len)
d, k, nf = F.shape
MX = make_mx_matrix(X, d, k, nf, n_len)
MX.shape
s1 = MX @ np.ravel(F, order='F')
s2 = MF @ x
np.all(s1 == s2)

True

In [36]:
X

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [61]:
##%%timeit
##MX = make_mx_matrix(X_input, d, k, nf, n_len)

In [62]:
##%%timeit
##MF = make_mf_matrix(F[0], n_len)

In [63]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def evaluate_classifier(X_batch, MFs, W):
    s1 = MFs[0] @ X_batch
    X1_batch = np.maximum(np.zeros_like(s1), s1)
    s2 = MFs[1] @ X1_batch
    X2_batch = np.maximum(np.zeros_like(s2), s2)
    S_batch = W@X2_batch
    P_batch = softmax(S_batch)
    return X1_batch, X2_batch, P_batch

def compute_loss(X_batch, Y_batch, MFs, W):
    _, _, P_batch = evaluate_classifier(X_batch, MFs, W)
    fact = 1/X_batch.shape[1]
    lcross_sum = np.sum(np.diag(-Y_batch.T@np.log(P_batch)))
    return fact*lcross_sum

In [64]:
MFs = [make_mf_matrix(F[0], n_len), make_mf_matrix(F[1], n_len1)]
_, _, P = evaluate_classifier(X_train, MFs, W)
P.shape

(18, 19834)

In [65]:
X_batch = X_train[:,:100]
Y_batch = Y_train[:,:100]
MFs = [make_mf_matrix(F[0], n_len), make_mf_matrix(F[1], n_len1)]

In [66]:
def compute_gradients(X_batch, Y_batch, MFs, W):
    X1_batch, X2_batch, P_batch = evaluate_classifier(X_batch, MFs, W)
    n = X_batch.shape[1]
    fact = 1/n
    G_batch = -(Y_batch - P_batch)
    dW = fact*(G_batch@X2_batch.T)
    G_batch = W.T @ G_batch
    G_batch = G_batch * (X2_batch > 0)
    g_0 = G_batch[:, 0]
    x_0 = X1_batch[:, 0]
    v = g_0.T @ make_mx_matrix(x_0, n1, k2, n2, n_len1)
    dF2 = fact*v
    for i in range(1, n):
        gi = G_batch[:, i]
        xi = X1_batch[:, i]
        v = gi.T @ make_mx_matrix(xi, n1, k2, n2, n_len1)
        dF2 += fact*v
    G_batch = MFs[1].T @ G_batch
    G_batch = G_batch * (X1_batch > 0)
    g_0 = G_batch[:, 0]
    x_0 = X_batch[:, 0]
    v = g_0.T @ make_mx_matrix(x_0, d, k1, n1, n_len)
    dF1 = fact*v
    for i in range(1, n):
        gi = G_batch[:, i]
        xi = X_batch[:, i]
        v = gi.T @ make_mx_matrix(xi, d, k1, n1, n_len)
        dF1 += fact*v
    return dW, dF1.reshape(d, k1, n1), dF2.reshape(n1, k2, n2)   

In [68]:
import numpy as np

ConvNet = {'F': F, 'W': W}
MFs = [make_mf_matrix(F[0], n_len), make_mf_matrix(F[1], n_len1)]

def NumericalGradient(X_inputs, Ys, ConvNet, h):
    try_ConvNet = ConvNet.copy()
    Gs = [None] * (len(ConvNet['F']) + 1)  # Create a list to store gradients for each F and W

    # Compute gradients for convolutional layers
    for l in range(len(ConvNet['F'])):
        try_ConvNet['F'][l] = np.array(ConvNet['F'][l], copy=True)
        
        Gs[l] = np.zeros_like(ConvNet['F'][l])
        nf = ConvNet['F'][l].shape[2]
        
        for i in range(nf):
            try_ConvNet['F'][l] = np.array(ConvNet['F'][l], copy=True)
            F_try = np.squeeze(ConvNet['F'][l][:, :, i])
            G = np.zeros(np.prod(F_try.shape))
            
            for j in range(len(G)):
                F_try1 = np.array(F_try, copy=True)
                F_try1.flat[j] = F_try.flat[j] - h
                try_ConvNet['F'][l][:, :, i] = F_try1.reshape(F_try.shape)
                
                l1 = compute_loss(X_inputs, Ys, MFs, try_ConvNet['W'])
                
                F_try2 = np.array(F_try, copy=True)
                F_try2.flat[j] = F_try.flat[j] + h
                try_ConvNet['F'][l][:, :, i] = F_try2.reshape(F_try.shape)
                
                l2 = compute_loss(X_inputs, Ys, MFs, try_ConvNet['W'])
                
                G[j] = (l2 - l1) / (2 * h)
                try_ConvNet['F'][l][:, :, i] = F_try  # Reset to original F
            
            Gs[l][:, :, i] = G.reshape(F_try.shape)
    
    # Compute the gradient for the fully connected layer
    W_try = ConvNet['W']
    G = np.zeros(np.prod(W_try.shape))
    for j in range(len(G)):
        W_try1 = np.array(W_try, copy=True)
        W_try1.flat[j] = W_try.flat[j] - h
        try_ConvNet['W'] = W_try1
        
        l1 = compute_loss(X_inputs, Ys, MFs, try_ConvNet['W'])
        
        W_try2 = np.array(W_try, copy=True)
        W_try2.flat[j] = W_try.flat[j] + h
        try_ConvNet['W'] = W_try2
        
        l2 = compute_loss(X_inputs, Ys, MFs, try_ConvNet)
        
        G[j] = (l2 - l1) / (2 * h)
        try_ConvNet['W'] = W_try  # Reset to original W
    
    Gs[-1] = G.reshape(W_try.shape)
    
    return Gs
